In [1]:
import google.generativeai as genai

d:\HERO\LLM\EnglishHindiTranslation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os

In [2]:
GOOGLE_API_KEY= os.environ["GOOGLE_API_KEY"]

genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [16]:
response = model.generate_content("Convert in english : वाह")

In [17]:
response.text

'"Wow" \n'

In [15]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [18]:
import pandas as pd

In [26]:
df = pd.read_csv("hin.txt" , sep = "\t" , header = None)
df.head()

,0,1,2
0,Wow!,वाह!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Duck!,झुको!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Duck!,बतख़!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Help!,बचाओ!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Jump.,उछलो.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [27]:
df = df.iloc[:,:2]

In [28]:
df.head()

,0,1
0,Wow!,वाह!
1,Duck!,झुको!
2,Duck!,बतख़!
3,Help!,बचाओ!
4,Jump.,उछलो.


In [29]:
df = df.rename(columns = {0 : "english" , 1 : "hindi"})

In [30]:
df.head()

,english,hindi
0,Wow!,वाह!
1,Duck!,झुको!
2,Duck!,बतख़!
3,Help!,बचाओ!
4,Jump.,उछलो.


In [31]:
df.to_csv("updated_data.csv" , index = False)

In [34]:
loader = CSVLoader(file_path="updated_data.csv" , source_column = "english")
data = loader.load()

In [35]:
data

[Document(page_content='english: Wow!\nhindi: वाह!', metadata={'source': 'Wow!', 'row': 0}),
 Document(page_content='english: Duck!\nhindi: झुको!', metadata={'source': 'Duck!', 'row': 1}),
 Document(page_content='english: Duck!\nhindi: बतख़!', metadata={'source': 'Duck!', 'row': 2}),
 Document(page_content='english: Help!\nhindi: बचाओ!', metadata={'source': 'Help!', 'row': 3}),
 Document(page_content='english: Jump.\nhindi: उछलो.', metadata={'source': 'Jump.', 'row': 4}),
 Document(page_content='english: Jump.\nhindi: कूदो.', metadata={'source': 'Jump.', 'row': 5}),
 Document(page_content='english: Jump.\nhindi: छलांग.', metadata={'source': 'Jump.', 'row': 6}),
 Document(page_content='english: Hello!\nhindi: नमस्ते।', metadata={'source': 'Hello!', 'row': 7}),
 Document(page_content='english: Hello!\nhindi: नमस्कार।', metadata={'source': 'Hello!', 'row': 8}),
 Document(page_content='english: Cheers!\nhindi: वाह-वाह!', metadata={'source': 'Cheers!', 'row': 9}),
 Document(page_content='en

In [37]:
c

d:\HERO\LLM\EnglishHindiTranslation\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents = data , embedding = embeddings)

In [39]:
vectordb.save_local("faiss_index")

In [40]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents("i like your hat.")
rdocs

d:\HERO\LLM\EnglishHindiTranslation\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='english: This hat does match the brown dress.\nhindi: यह टोपी उस ब्राऊन ड्रेस के साथ जचती नहीं है।', metadata={'source': 'This hat does match the brown dress.', 'row': 2240}),
 Document(page_content='english: Take off your hat when you enter a classroom.\nhindi: क्लासरूम में घुसते समय अपनी टोपी उतारा करो।', metadata={'source': 'Take off your hat when you enter a classroom.', 'row': 2714}),
 Document(page_content='english: He has a hat on.\nhindi: उसने टोपी पहनी हुई है।', metadata={'source': 'He has a hat on.', 'row': 220}),
 Document(page_content="english: That's too small to fit on your head.\nhindi: वह तुम्हारे सर के लिए बहुत छोटा है।", metadata={'source': "That's too small to fit on your head.", 'row': 2302})]

In [41]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following sentence in english, generate an new sentence in hindi based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [45]:
PROMPT

PromptTemplate(input_variables=['context', 'question'], template='Given the following sentence in english, generate an new sentence in hindi based on this context only.\nIn the answer try to provide as much text as possible from "response" section in the source document context without making much changes.\nDon\'t try to make up an answer.\n\nCONTEXT: {context}\n\nQUESTION: {question}')

In [43]:
from langchain.chains import RetrievalQA

# chain = RetrievalQA.from_chain_type(llm = model , chain_type="stuff" , retriever = retriever , input_key = "query" , return_source_documents = True , chain_type_kwargs=chain_type_kwargs)

# chain("i like your hat.")

In [50]:
chain = RetrievalQA.from_chain_type(llm = llm , chain_type="stuff" , retriever = retriever , input_key = "query" , return_source_documents = True , chain_type_kwargs=chain_type_kwargs)

In [47]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [49]:
llm = ChatGoogleGenerativeAI(model="gemini-pro" , google_api_key = GOOGLE_API_KEY)
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

**Verse 1**
In realms of code, where thoughts reside,
A tale unfolds, of LangChain's stride.
A model vast, with boundless might,
A beacon of progress, shining bright.

**Chorus**
Oh, LangChain, LangChain, a wonder to behold,
Unveiling secrets, stories yet untold.
From text to code, a seamless blend,
Transforming dreams, transcending the end.

**Verse 2**
With every query, a symphony of words,
LangChain weaves magic, like a flock of birds.
Summaries, translations, tasks it can do,
A tireless servant, always true.

**Verse 3**
In research labs and classrooms wide,
LangChain empowers, with knowledge as its guide.
Students and scholars, eager to explore,
Unlock the mysteries, forevermore.

**Chorus**
Oh, LangChain, LangChain, a wonder to behold,
Unveiling secrets, stories yet untold.
From text to code, a seamless blend,
Transforming dreams, transcending the end.

**Verse 4**
As technology advances, LangChain will soar,
A guiding star, illuminating every shore.
In a world of data, where kno

In [51]:
chain("I love my hat")

d:\HERO\LLM\EnglishHindiTranslation\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'I love my hat',
 'result': 'मुझे मेरी टोपी बहुत पसंद है।',
 'source_documents': [Document(page_content='english: This hat does match the brown dress.\nhindi: यह टोपी उस ब्राऊन ड्रेस के साथ जचती नहीं है।', metadata={'source': 'This hat does match the brown dress.', 'row': 2240}),
  Document(page_content='english: Take off your hat when you enter a classroom.\nhindi: क्लासरूम में घुसते समय अपनी टोपी उतारा करो।', metadata={'source': 'Take off your hat when you enter a classroom.', 'row': 2714}),
  Document(page_content='english: He has a hat on.\nhindi: उसने टोपी पहनी हुई है।', metadata={'source': 'He has a hat on.', 'row': 220}),
  Document(page_content="english: That's too small to fit on your head.\nhindi: वह तुम्हारे सर के लिए बहुत छोटा है।", metadata={'source': "That's too small to fit on your head.", 'row': 2302})]}

In [53]:
chain("i do not know how to make a chapatii")

{'query': 'i do not know how to make a chapatii',
 'result': 'मुझे नहीं पता कि चपाती कैसे बनाई जाती है।',
 'source_documents': [Document(page_content="english: I haven't eaten yet.\nhindi: मैंने अभी तक नहीं खाया है।", metadata={'source': "I haven't eaten yet.", 'row': 515}),
  Document(page_content="english: I don't feel like eating now.\nhindi: मुझे अभी खाना खाने का मन नहीं है।", metadata={'source': "I don't feel like eating now.", 'row': 1498}),
  Document(page_content='english: Tom is making boiled eggs.\nhindi: टॉम उबले अंडे बना रहा है।', metadata={'source': 'Tom is making boiled eggs.', 'row': 1202}),
  Document(page_content="english: The food's not ready yet.\nhindi: खाना अभी तक तैयार नहीं हुआ है।", metadata={'source': "The food's not ready yet.", 'row': 1054})]}